In [1]:
import lib
import numpy as np
from scipy import special
import os

# Hartree Fock computation of the ground state of Quantum Dots

In this Jupyter Notebook we are going to compare the groundstate energies obtained with the analytical formula for the two body integrals with Montecarlo integration. In both cases, the Self-Consistent Field method is used.

### Basis functions
First we write the basis we are going to use, i.e., the 3D eigenfunctions of the harmonic oscillator.

In [2]:
lz = 0.5
lx = 0.75

OMEGA_X = 1/lx**2
OMEGA_Z = 1/lz**2

def Hermite_pol(x, n):
    """
    Evaluates the Hermite polynomial of order n at x.

    Parameters
    ==========
    x : float or np.ndarray
        Values in which to evaluate the Hermite polynomial
    n : int
        Oder of the Hermite polynomial

    Returns
    =======
    float or np.ndarray
    """
    return special.eval_hermite(n, x)

def HO_wf(x, n, omega=OMEGA_X):
    """
    Evaluates the n^th eigenfunction of the Harmonic Oscillator at x.
    The units of x are sqrt(hbar/m*omega). 

    Parameters
    ==========
    x : float or np.ndarray(N)
        Values in which to evaluate eigenfunction
    n : int
        Number of the eigenfunction
    omega : float
        Angular frequency of the Harmonic Oscillator

    Returns
    =======
    float or np.ndarray(N)
    """
    #return omega**0.25 * Hermite_pol(np.sqrt(omega)*x, n) * np.exp(-omega*x**2 / 2) / np.sqrt(2**n * np.math.factorial(n) * np.sqrt(np.pi))
    return omega**0.25 * Hermite_pol(np.sqrt(omega)*x, n) / np.sqrt(2**n * np.math.factorial(n) * np.sqrt(np.pi))

def HO_wf_3D(x, y, z, nx, ny, nz, omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z):
    """
    Evaluates the n^th eigenfunction of the (anisotropic) Harmonic Oscillator at x,y,z.
    The units of x,y,z are sqrt(hbar/m*omega). 

    Parameters
    ==========
    x, y, z : float or np.ndarray(N)
        Position in which to evaluate the wave function
    nx, ny, nz : int
        Number of the eigenfunction for each cartesian coordinate
    omega_x, omega_y, omega_z : float
        Harmonic osciallation constant for each cartesian coordinate

    Returns
    =======
    float or np.ndarray(N)
    """
    return HO_wf(x, nx, omega=omega_x)*HO_wf(y, ny, omega=omega_y)*HO_wf(z, nz, omega=omega_z)

def index_to_q_numbers(k):
    """
    Returns the quantum numbers nx, ny, nz associated with the basis index k

    Parameters
    ----------
    k: int
        Index of the basis from 1 to 14

    Returns
    ----------
    nz ,ny, nz : int 
        Quantum numbers
    """

    q_numbers = np.array([(0,0,0),(0,0,1),(0,1,0),(1,0,0),(0,1,1),(1,0,1),(1,1,0),(1,1,1),(1,1,2),(1,2,1),(2,1,1),(1,2,2),(2,1,2),(2,2,1)])

    return q_numbers[k]

We now write the functions to calculate the integrands of the two-body integrals

In [3]:
def integrand_2(R, p, r, q, s):
    r1 = R[:,0:3]
    r2 = R[:,3:6]
    r12 = np.sqrt(np.sum((r1 - r2)**2, axis=-1))
    
    x1 = R[:,0]
    y1 = R[:,1]
    z1 = R[:,2]
    x2 = R[:,3]
    y2 = R[:,4]
    z2 = R[:,5]
    
    n_p = index_to_q_numbers(p-1)
    n_r = index_to_q_numbers(r-1)
    n_q = index_to_q_numbers(q-1)
    n_s = index_to_q_numbers(s-1)
    
    result = 1/r12
    #print("1", result)
    result *= HO_wf_3D(x1, y1, z1, n_p[0], n_p[1], n_p[2], omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z)
    #print("2", result)
    result *= HO_wf_3D(x2, y2, z2, n_r[0], n_r[1], n_r[2], omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z)
    #print("3", result)
    result *= HO_wf_3D(x1, y1, z1, n_q[0], n_q[1], n_q[2], omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z)
    #print("4", result)
    result *= HO_wf_3D(x2, y2, z2, n_s[0], n_s[1], n_s[2], omega_x=OMEGA_X, omega_y=OMEGA_X, omega_z=OMEGA_Z)
    #print("5", result)
    '''
    result *= np.exp(OMEGA_X*x1**2/2)*np.exp(OMEGA_X*x1**2/2)*np.exp(OMEGA_Z*x1**2/2)
    result *= np.exp(OMEGA_X*x1**2/2)*np.exp(OMEGA_X*x1**2/2)*np.exp(OMEGA_Z*x1**2/2)
    result *= np.exp(OMEGA_X*x1**2/2)*np.exp(OMEGA_X*x1**2/2)*np.exp(OMEGA_Z*x1**2/2)
    result *= np.exp(OMEGA_X*x1**2/2)*np.exp(OMEGA_X*x1**2/2)*np.exp(OMEGA_Z*x1**2/2)
    '''
    
    return result

def norm_product(p, r, q, s):
    return (np.sqrt(2*np.pi/OMEGA_X))**8 * (np.sqrt(2*np.pi/OMEGA_Z))**4

def two_body_integrand(p, r, q, s):
    f = lambda R: integrand_2(R, p, r, q, s)*norm_product(p, r, q, s)
    
    return f

### One- and two- body integral computations
We first initialize some necessary parameters

In [4]:
N_electrons = 2
N_basis = 14
N_points = 10000
integrals_file = "integrals_QD.npy"

normalized_wf = True

max_iter_SCF = 500
eps_SCF = 1e-5
Delta_SCF = 0

The exact one-body integral function is

In [5]:
def analytical_1(p,q):
    np = index_to_q_numbers(p-1)
    I = (p==q)*(OMEGA_X*(np[0] + np[1] + 1) + OMEGA_Z*(np[2] + 0.5))
    
    return I

The covariance function for the exponents of the gaussians is

In [6]:
def f_cov(p, r, q, s):
    cov = np.diag([1/OMEGA_X, 1/OMEGA_X, 1/OMEGA_Z, 1/OMEGA_X, 1/OMEGA_X, 1/OMEGA_Z])
    
    return cov

We then calculate and store the one- and two-body integrals using lib.py

In [7]:
"""
# Este bloque habrá que acabar quitándolo
cov = f_cov(1,1,1,1)
integrand = two_body_integrand(1,1,1,1)

dim = cov.shape[0]
steps = np.random.multivariate_normal(np.zeros(dim), cov, N_points) 

I = integrand(np.array([[ 2.16448580e-01,  5.91984420e-01,  4.11097144e-01, -1.45209680e-04,
  -3.64044792e-01,  8.23999340e-02]]))
I_avg= np.average(integrand(steps))
print(norm_product(1,1,1,1))
print(integrand_2(np.array([[ 2.16448580e-01,  5.91984420e-01,  4.11097144e-01, -1.45209680e-04,
  -3.64044792e-01,  8.23999340e-02]]), 1,1,1,1))
print(I)
"""

'\n# Este bloque habrá que acabar quitándolo\ncov = f_cov(1,1,1,1)\nintegrand = two_body_integrand(1,1,1,1)\n\ndim = cov.shape[0]\nsteps = np.random.multivariate_normal(np.zeros(dim), cov, N_points) \n\nI = integrand(np.array([[ 2.16448580e-01,  5.91984420e-01,  4.11097144e-01, -1.45209680e-04,\n  -3.64044792e-01,  8.23999340e-02]]))\nI_avg= np.average(integrand(steps))\nprint(norm_product(1,1,1,1))\nprint(integrand_2(np.array([[ 2.16448580e-01,  5.91984420e-01,  4.11097144e-01, -1.45209680e-04,\n  -3.64044792e-01,  8.23999340e-02]]), 1,1,1,1))\nprint(I)\n'

In [8]:
# One- and Two-body integrals
MC_args = {"f_cov":f_cov, "f_integrand":two_body_integrand, "N_points":N_points}
integrals = lib.integral_master(N_basis)
integrals.calculate(integrals_file, analytical_1, analytical_2 = None, MC_args = MC_args)

We calculate the overlap matrix S

In [9]:
if not normalized_wf:
    S = np.zeros((4,4))
    for p in range(4):
        for q in range(4):
            S[p][q] = (np.pi/(ALPHA[p]+ALPHA[q]))**(1.5)
    SVAL, SVEC = np.linalg.eigh(S) 
    SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
    X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
else:
    S = np.eye(N_basis)
# print('S',S)

Finally, we run the Self-Consistent Field for Hartree Fock

In [10]:
lib.SCF(N_electrons, integrals, S, max_iter_SCF, eps_SCF)

E = 5.4648997 | N(SCF) = 1
E = -1399.9358938 | N(SCF) = 2
E = 4315.0582658 | N(SCF) = 3
E = -98026.8491833 | N(SCF) = 4
E = -257700.3077554 | N(SCF) = 5
E = -220213.2821019 | N(SCF) = 6
E = -349052.4651183 | N(SCF) = 7
E = -373146.4910527 | N(SCF) = 8
E = -381426.9448760 | N(SCF) = 9
E = -392258.2597912 | N(SCF) = 10
E = -405610.3442223 | N(SCF) = 11
E = -415240.9137380 | N(SCF) = 12
E = -415870.4916912 | N(SCF) = 13
E = -416628.7856821 | N(SCF) = 14
E = -418101.5045262 | N(SCF) = 15
E = -419805.6505315 | N(SCF) = 16
E = -424094.5344052 | N(SCF) = 17
E = -399467.1321458 | N(SCF) = 18
E = -409258.5959739 | N(SCF) = 19
E = -409847.2482987 | N(SCF) = 20
E = -412342.9923892 | N(SCF) = 21
E = -416841.5122445 | N(SCF) = 22
E = -419179.5653152 | N(SCF) = 23
E = -420244.3833463 | N(SCF) = 24
E = -414669.2615343 | N(SCF) = 25
E = -416701.6054470 | N(SCF) = 26
E = -418133.0166459 | N(SCF) = 27
E = -418319.0963208 | N(SCF) = 28
E = -418281.2249864 | N(SCF) = 29
E = -418584.9166421 | N(SCF) = 30
E

E = -410859.4458230 | N(SCF) = 241
E = -415937.7154399 | N(SCF) = 242
E = -416010.1696147 | N(SCF) = 243
E = -417486.2510461 | N(SCF) = 244
E = -422580.4730352 | N(SCF) = 245
E = -390991.2816220 | N(SCF) = 246
E = -446983.8174234 | N(SCF) = 247
E = -278567.8589414 | N(SCF) = 248
E = -316292.2544742 | N(SCF) = 249
E = -361623.0401938 | N(SCF) = 250
E = -391208.5613833 | N(SCF) = 251
E = -408361.0460579 | N(SCF) = 252
E = -413059.5128669 | N(SCF) = 253
E = -413946.3705882 | N(SCF) = 254
E = -416402.0199865 | N(SCF) = 255
E = -418297.3174516 | N(SCF) = 256
E = -419973.5852362 | N(SCF) = 257
E = -441123.6262700 | N(SCF) = 258
E = -391400.8621771 | N(SCF) = 259
E = -398956.7096387 | N(SCF) = 260
E = -411963.6335398 | N(SCF) = 261
E = -427058.4167366 | N(SCF) = 262
E = -382018.3729964 | N(SCF) = 263
E = -396088.1273105 | N(SCF) = 264
E = -409574.1050471 | N(SCF) = 265
E = -416311.5041033 | N(SCF) = 266
E = -426430.0076618 | N(SCF) = 267
E = -393238.5522349 | N(SCF) = 268
E = -396770.4297260 

KeyboardInterrupt: 